In [1]:
import streamlit as st
from langchain.chains import ConversationalRetrievalChain
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.llms import LlamaCpp
import os
import tempfile
from multiprocessing import Pool

In [2]:
# Function to process PDF files and extract text
def process_pdf_files(uploaded_files):
    text = []
    for file in uploaded_files:
        file_extension = os.path.splitext(file.name)[1]
        with tempfile.NamedTemporaryFile(delete=False) as temp_file:
            temp_file.write(file.read())
            temp_file_path = temp_file.name
        loader = None
        if file_extension == ".pdf":
            loader = PyPDFLoader(temp_file_path)
        if loader:
            text.extend(loader.load())
            os.remove(temp_file_path)
    return text

In [3]:
# Function to initialize session state variables
def initialize_session_state():
    if 'history' not in st.session_state:
        st.session_state['history'] = []
    if 'generated' not in st.session_state:
        st.session_state['generated'] = ["Hello! Ask me anything about 🤗"]
    if 'past' not in st.session_state:
        st.session_state['past'] = ["Hey! 👋"]

In [4]:
# Function to handle conversation with the LlamaCpp model
def conversation_chat(query, chain, history):
    result = chain.invoke({"question": query, "chat_history": history})
    history.append((query, result["answer"]))
    return result["answer"]

In [5]:
# Function to display chat history in the Streamlit UI
def display_chat_history(chain):
    reply_container = st.container()
    container = st.container()
    with container:
        with st.form(key='my_form', clear_on_submit=True):
            user_input = st.text_input("Question:", placeholder="Ask about your PDF", key='input')
            submit_button = st.form_submit_button(label='Send')
        if submit_button and user_input:
            with st.spinner('Generating response...'):
                output = conversation_chat(user_input, chain, st.session_state['history'])
            st.session_state['past'].append(user_input)
            st.session_state['generated'].append(output)
    if st.session_state['generated']:
        with reply_container:
            for i in range(len(st.session_state['generated'])):
                st.markdown(f'<span style="font-size: 2em;">😎</span>: {st.session_state["past"][i]}', unsafe_allow_html=True)
                st.markdown(f'<span style="font-size: 2em;">🤖</span>: {st.session_state["generated"][i]}', unsafe_allow_html=True)
                st.markdown("---")

In [6]:
# Function to create the conversational chain
def create_conversational_chain(text_chunks):
    llm = LlamaCpp(
        streaming=True,
        model_path="models/mistral-7b-instruct-v0.1.Q4_K_S.gguf",
        temperature=0.75,
        top_p=1,
        verbose=True,
        n_ctx=4096,
        n_gpu_layers=-1, 
        n_batch=512 
    )
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'cpu'} 
    )
    vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)
    chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type='stuff',
        retriever=vector_store.as_retriever(search_kwargs={"k": 2}),
        memory=memory
    )
    return chain

In [7]:
# Main function to run the Streamlit app
def main():
    initialize_session_state()
    st.title("Almeno Chatbot")
    st.sidebar.title("Document Processing")
    uploaded_files = st.sidebar.file_uploader("Upload files", accept_multiple_files=True)
    if uploaded_files:
        with st.spinner("Processing the files..."):
            with Pool() as pool:
                text_chunks = pool.map(process_pdf_files, [uploaded_files])[0]
        chain = create_conversational_chain(text_chunks)
        display_chat_history(chain)

In [8]:
if __name__ == '__main__':
    main()

2024-11-21 17:22:20.079 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-21 17:22:20.080 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2024-11-21 17:22:20.080 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-21 17:22:20.080 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-21 17:22:20.081 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-21 17:22:20.081 WARNING streamlit.runtime.scriptrunner_utils.script_run_c

In [11]:
!streamlit run --/Users/krishnarajput/Documents/AI/almeno-content-engine/app.py

Usage: streamlit run [OPTIONS] TARGET [ARGS]...

  Run a Python script, piping stderr to Streamlit.

  The script can be local or it can be an url. In the latter case, Streamlit
  will download the script to a temporary file and runs this file.

Options:
  --global.disableWidgetStateDuplicationWarning BOOLEAN
                                  By default, Streamlit displays a warning
                                  when a user sets both a widget default value
                                  in the function defining the widget and a
                                  widget value via the widget's key in
                                  `st.session_state`.
                                  
                                  If you'd like to turn off this warning, set
                                  this to True.  [env var: STREAMLIT_GLOBAL_DI
                                  SABLE_WIDGET_STATE_DUPLICATION_WARNING]
  --global.showWarningOnDirectExecution BOOLEAN
                    